In [33]:
import json
import pandas as pd
import time
import gcsfs


In [34]:
from google.cloud import storage 
from google.cloud import bigquery as bq
from pathlib import Path
import os

In [35]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'peii-242723-111b55a1e645.json'

In [119]:
df = pd.read_csv('gs://impact-index-shared-resources/data/interim/2019-11-06/interim_entrepreneur_data.csv')
df

,name,company_name,crunchbase_uuid,crunchbase_permalink,funding,jobs_created,patents,ipo,city_and_state,metro,gender,lgbtq,race_ethnicity,startout_id
0,Scott Brown,ColdSpark,226a627bc92415995985cbd94743276b,coldspark,6500000,1000.0,0,False,"Broomfield, CO","Denver-Aurora-Lakewood, CO",NaN,NaN,NaN,NaN
1,Robin Horwitz,Convo Communications,7d0675cefac592615e1cbb6c29fd403a,convo-communications,500000,100.0,0,False,"Austin, TX","Austin-Round Rock, TX",NaN,NaN,NaN,NaN
2,Jiren Parikh,SnapOne,ed5f8110a213395ececfe94660c0f602,snapone-inc,0,100.0,1,False,"Princeton, NJ","Trenton-Ewing, NJ",Male,NaN,NaN,NaN
3,Matthew Slipper,Symphony Communication Services,5f89826c5031a1932f27525b505b0a7f,symphony-3,461000000,500.0,1,False,"Palo Alto, CA","San Jose-Sunnyvale-Santa Clara, CA",Male,NaN,NaN,NaN
4,Ric Zhou,Kika Tech,437121710de6c1d5e8f8ebe555749fa8,kika-tech,63000000,500.0,0,False,"San Jose, CA","San Jose-Sunnyvale-Santa Clara, CA",NaN,NaN,NaN,NaN
5,Kumaran Thillainadarajah,Smart Skin Technologies,5974376ec194ff0e4322caded8d049d5,smart-skin-technologies,11545000,50.0,2,False,"New Brunswick, NJ","New York-Northern New Jersey-Long Island, NY-N...",Male,NaN,NaN,NaN
6,Sunil Agrawal,Armor5,3fca9b4b44b3ed61719f56c19af7387d,armor5,2000000,50.0,7,False,"Santa Clara, CA","San Jose-Sunnyvale-Santa Clara, CA",Male,NaN,NaN,NaN
7,Suzy Batiz,Poo~Pourri,3ac4350d0bd841a98beb24341e43a63f,poo-pourri,0,50.0,1,False,"Addison, TX","Dallas-Fort Worth-Arlington, TX",NaN,NaN,NaN,NaN
8,Brian Hastings,Gearbox Express,d4ed79c1d290badf8165832a8eed0d41,gearbox-express,2465000,50.0,0,False,"Mukwonago, WI","Milwaukee-Waukesha-West Allis, WI",Male,NaN,NaN,NaN
9,Brian Petersen,Podimetrics,8cded10bbd3be6f33dcf72367e5fb604,podimetrics,15900000,50.0,5,False,"Somerville, MA","Boston-Cambridge-Newton, MA-NH",NaN,NaN,NaN,NaN


In [106]:
PROJ_ROOT = Path().resolve().parent
KEYS_DIR = PROJ_ROOT / 'keys' 
keys = KEYS_DIR / 'Keys for Big Query Storage Admin - PEII.json'

In [107]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = str(keys)

In [108]:
client = bq.Client()

names_query="""
#standardSQL
SELECT gender, name FROM `bigquery-public-data.usa_names.usa_1910_current` LIMIT 1000
"""

job_config = bq.QueryJobConfig()
now=time.time()
query_job=client.query(names_query,location='US')
res=query_job.result()
print('query took:',round(time.time()-now,2),'s')

query took: 2.1 s


In [100]:
now=time.time()
social_security_df=res.to_dataframe()
print('load to pandas from BQ took:',round(time.time()-now,2),'s')

load to pandas from BQ took: 0.53 s


In [101]:
social_security_df = social_security_df.set_index('name')

In [103]:
social_security_df[social_security_df['gender'] == 'F']

,gender
name,
Mary,F
Helen,F
Dorothy,F
Ruth,F
Margaret,F
Mildred,F
Anna,F
Florence,F
Elizabeth,F


In [ ]:
df.name.astype(str)

for i in df.index:
    name = df.at[i, 'name'].split(' ')[0]
    for j in social_security_df.index:
        social_security_name = social_security_df.at[j, 'name']
        if name == social_security_name:
            print('true')

In [82]:
ss_dict = social_security_df.to_dict('split')
ss_names_lst = ss_dict['index']

In [84]:
ss_names_dict = {}


for i in range(len(ss_names_lst)):
    name = ss_names_lst[i]
    ss_names_dict[name] = 'f'
    
ss_names_dict.keys()

dict_keys(['Mary', 'Helen', 'Dorothy', 'Ruth', 'Margaret', 'Mildred', 'Anna', 'Florence', 'Elizabeth', 'Marie', 'Frances', 'Virginia', 'Thelma', 'Edna', 'Alice', 'Evelyn', 'Ethel', 'Gladys', 'Hazel', 'Martha', 'Catherine', 'Rose', 'Grace', 'Lillian', 'Esther', 'Clara', 'Edith', 'Gertrude', 'Irene', 'Louise', 'Lucille', 'Josephine', 'Eleanor', 'Pauline', 'Mabel', 'Kathryn', 'Elsie', 'Ann', 'Doris', 'Emma', 'Bertha', 'Katherine', 'Beatrice', 'Nellie', 'Betty', 'Julia', 'Marjorie', 'Bernice', 'Sarah', 'Agnes', 'Alma', 'Laura', 'Marian', 'Leona', 'Eva', 'Jean', 'Viola', 'Anne', 'Pearl', 'Ida', 'Jane', 'Stella', 'Bessie', 'Charlotte', 'Goldie', 'Myrtle', 'Violet', 'Lois', 'Wilma', 'Blanche', 'Vera', 'Marguerite', 'Erma', 'Jessie', 'Marion', 'Mae', 'Genevieve', 'Norma', 'Velma', 'Jennie', 'Geraldine', 'Barbara', 'Minnie', 'Olive', 'Opal', 'Ruby', 'Marcella', 'Theresa', 'Ada', 'Ellen', 'Beulah', 'Freda', 'Hilda', 'Sylvia', 'Harriet', 'Lena', 'Loretta', 'Alberta', 'Cora', 'Audrey', 'Georgia', 

In [ ]:
for i in df.index:
    df.at[i, 'f_name'] = 

In [120]:
#creating the first name column
for i in df.index:
    df.at[i, 'f_name'] = df.at[i, 'name'].split(' ')[0]

In [121]:
df

,name,company_name,crunchbase_uuid,crunchbase_permalink,funding,jobs_created,patents,ipo,city_and_state,metro,gender,lgbtq,race_ethnicity,startout_id,f_name
0,Scott Brown,ColdSpark,226a627bc92415995985cbd94743276b,coldspark,6500000,1000.0,0,False,"Broomfield, CO","Denver-Aurora-Lakewood, CO",NaN,NaN,NaN,NaN,Scott
1,Robin Horwitz,Convo Communications,7d0675cefac592615e1cbb6c29fd403a,convo-communications,500000,100.0,0,False,"Austin, TX","Austin-Round Rock, TX",NaN,NaN,NaN,NaN,Robin
2,Jiren Parikh,SnapOne,ed5f8110a213395ececfe94660c0f602,snapone-inc,0,100.0,1,False,"Princeton, NJ","Trenton-Ewing, NJ",Male,NaN,NaN,NaN,Jiren
3,Matthew Slipper,Symphony Communication Services,5f89826c5031a1932f27525b505b0a7f,symphony-3,461000000,500.0,1,False,"Palo Alto, CA","San Jose-Sunnyvale-Santa Clara, CA",Male,NaN,NaN,NaN,Matthew
4,Ric Zhou,Kika Tech,437121710de6c1d5e8f8ebe555749fa8,kika-tech,63000000,500.0,0,False,"San Jose, CA","San Jose-Sunnyvale-Santa Clara, CA",NaN,NaN,NaN,NaN,Ric
5,Kumaran Thillainadarajah,Smart Skin Technologies,5974376ec194ff0e4322caded8d049d5,smart-skin-technologies,11545000,50.0,2,False,"New Brunswick, NJ","New York-Northern New Jersey-Long Island, NY-N...",Male,NaN,NaN,NaN,Kumaran
6,Sunil Agrawal,Armor5,3fca9b4b44b3ed61719f56c19af7387d,armor5,2000000,50.0,7,False,"Santa Clara, CA","San Jose-Sunnyvale-Santa Clara, CA",Male,NaN,NaN,NaN,Sunil
7,Suzy Batiz,Poo~Pourri,3ac4350d0bd841a98beb24341e43a63f,poo-pourri,0,50.0,1,False,"Addison, TX","Dallas-Fort Worth-Arlington, TX",NaN,NaN,NaN,NaN,Suzy
8,Brian Hastings,Gearbox Express,d4ed79c1d290badf8165832a8eed0d41,gearbox-express,2465000,50.0,0,False,"Mukwonago, WI","Milwaukee-Waukesha-West Allis, WI",Male,NaN,NaN,NaN,Brian
9,Brian Petersen,Podimetrics,8cded10bbd3be6f33dcf72367e5fb604,podimetrics,15900000,50.0,5,False,"Somerville, MA","Boston-Cambridge-Newton, MA-NH",NaN,NaN,NaN,NaN,Brian


In [125]:
null = df[pd.isnull(df['gender'])]

In [114]:
count = 0

for i in df.index:
    name = df.at[i, 'f_name']
    if name in ss_names_dict.keys():
        df.at[i, 'gender'] = 'f'
        count += 1
        
print(count)

2624


In [126]:
count = 0

for i in null.index:
    name = null.at[i, 'f_name']
    if name in ss_names_dict.keys():
        df.at[i, 'gender'] = 'f'
        count += 1
        
print(count)

1181


In [115]:
df

,name,company_name,crunchbase_uuid,crunchbase_permalink,funding,jobs_created,patents,ipo,city_and_state,metro,gender,lgbtq,race_ethnicity,startout_id,f_name
0,Scott Brown,ColdSpark,226a627bc92415995985cbd94743276b,coldspark,6500000,1000.0,0,False,"Broomfield, CO","Denver-Aurora-Lakewood, CO",NaN,NaN,NaN,NaN,Scott
1,Robin Horwitz,Convo Communications,7d0675cefac592615e1cbb6c29fd403a,convo-communications,500000,100.0,0,False,"Austin, TX","Austin-Round Rock, TX",NaN,NaN,NaN,NaN,Robin
2,Jiren Parikh,SnapOne,ed5f8110a213395ececfe94660c0f602,snapone-inc,0,100.0,1,False,"Princeton, NJ","Trenton-Ewing, NJ",Male,NaN,NaN,NaN,Jiren
3,Matthew Slipper,Symphony Communication Services,5f89826c5031a1932f27525b505b0a7f,symphony-3,461000000,500.0,1,False,"Palo Alto, CA","San Jose-Sunnyvale-Santa Clara, CA",Male,NaN,NaN,NaN,Matthew
4,Ric Zhou,Kika Tech,437121710de6c1d5e8f8ebe555749fa8,kika-tech,63000000,500.0,0,False,"San Jose, CA","San Jose-Sunnyvale-Santa Clara, CA",NaN,NaN,NaN,NaN,Ric
5,Kumaran Thillainadarajah,Smart Skin Technologies,5974376ec194ff0e4322caded8d049d5,smart-skin-technologies,11545000,50.0,2,False,"New Brunswick, NJ","New York-Northern New Jersey-Long Island, NY-N...",Male,NaN,NaN,NaN,Kumaran
6,Sunil Agrawal,Armor5,3fca9b4b44b3ed61719f56c19af7387d,armor5,2000000,50.0,7,False,"Santa Clara, CA","San Jose-Sunnyvale-Santa Clara, CA",Male,NaN,NaN,NaN,Sunil
7,Suzy Batiz,Poo~Pourri,3ac4350d0bd841a98beb24341e43a63f,poo-pourri,0,50.0,1,False,"Addison, TX","Dallas-Fort Worth-Arlington, TX",NaN,NaN,NaN,NaN,Suzy
8,Brian Hastings,Gearbox Express,d4ed79c1d290badf8165832a8eed0d41,gearbox-express,2465000,50.0,0,False,"Mukwonago, WI","Milwaukee-Waukesha-West Allis, WI",Male,NaN,NaN,NaN,Brian
9,Brian Petersen,Podimetrics,8cded10bbd3be6f33dcf72367e5fb604,podimetrics,15900000,50.0,5,False,"Somerville, MA","Boston-Cambridge-Newton, MA-NH",NaN,NaN,NaN,NaN,Brian


In [59]:
df[df.f_name.isnull() == True]

,name,company_name,crunchbase_uuid,crunchbase_permalink,funding,jobs_created,patents,ipo,city_and_state,metro,gender,lgbtq,race_ethnicity,startout_id,f_name


In [ ]:
for i in df.index:
    first = df.at[i, 'f_name']
    if 

In [ ]:
name_gender = {}

for i in social_security_df.index:
    social_security_name = social_security_df.at[i, 'name']
    social_security_gender = social_security_df.at[i, 'name']
    name_gender[social]
    for j in social_security_df.index:
        
        if name == social_security_name:
            print('true')

In [5]:
project = 'peii-242723'
dataset_id = 'entrepre'
table_id = 'Combined_tables_39k'
bucket_name = 'muse-data'

# Setup Storage vars
storage_client = storage.Client(project=project)
bucket = storage_client.get_bucket('muse-data')

# Setup BigQuery vars
bq_client = bigquery.Client()
dataset_ref = bq_client.dataset(dataset_id, project=project)
table_ref = dataset_ref.table(table_id)

#destination
destination_uri = "gs://{}/{}".format(bucket_name, "combined_tables_raw.json")

In [7]:
#set job configuration to extract job as JSON files
job_config = bigquery.ExtractJobConfig()
job_config.destination_format = 'NEWLINE_DELIMITED_JSON'
job_config.write_disposition = 'WRITE_TRUNCATE'

In [8]:
extract_job = bq_client.extract_table(
    table_ref,
    destination_uri,
    # Location must match that of the source table.
    location="US",
    job_config=job_config
)  # API request
extract_job.result()  # Waits for job to complete.

print(
    "Exported {}:{}.{} to {}".format(project, dataset_id, table_id, destination_uri)
)

Exported smart-science:Learners_Questions.Combined_Tables to gs://muse-data/combined_tables_raw.json


In [9]:
df = pd.read_json(destination_uri, lines=True)

In [10]:
df.shape

(32687, 11)

In [11]:
df

,answer,answer_type,last_modified,learner,learner_name,metatag,name,question,tag,text,who
0,NaN,binary,2017-06-04 15:56:53.69455 UTC,4535845151309824,Carolina,[answer],UFqE8BiENzhk07YcA0oD8Eaxrb50LSts,368,[yes],Do you have reliable childcare for {kid_name}?,parent
1,NaN,binary,2017-06-04 15:56:57.46436 UTC,4535845151309824,Carolina,[answer],kLHlljhhWAnDEAixy2iaR9QkNhPYgdiw,371,[yes],Do you work outside of the home?,parent
2,NaN,binary,2017-08-31 11:44:55.22537 UTC,4535845151309824,Carolina,[answer],AI9iuqtEMakSt0EqPasH4B9MxQuDgice,38,[definitely],Did you graduate from high school?,parent
3,1.0,binary,2017-01-15 01:09:13.02664 UTC,4536217521618944,Katie,[answer],zUXTth86bHSuv5tzk3HZP3pyt2cj7Zhp,22,[yes],Do you know where the nearest playground is fr...,parent
4,1.0,binary,2017-01-15 01:11:47.43045 UTC,4536217521618944,Katie,[answer],cjTk7pkKA37b4TAOuDl1gO5n0JUxOW4f,22,[yes],Do you know where the nearest playground is fr...,parent
5,1.0,scale,2017-01-15 01:09:10.63448 UTC,4536217521618944,Katie,[answer],fF1jcJpeFa0QNrMZDLDDYfvHT3iT5OeD,236,[yes],"Does {kid_name} complete familiar books, espec...",child
6,1.0,scale,2017-01-15 01:11:49.84119 UTC,4536217521618944,Katie,[answer],iFDQCB2EFajl0ZOJRWxVFb2Stg7trWHb,283,[yes],Does {kid_name} enjoy trying new things?,child
7,1.0,scale,2017-01-15 01:08:00.33935 UTC,4536217521618944,Katie,[answer],qiLwtrCvLnM6nDyuHMyV0SieyuYxFtlq,285,[yes],Does {kid_name} enjoy creating elaborate prete...,child
8,0.0,binary,2017-01-15 01:13:38.63336 UTC,4536217521618944,Katie,[answer],P5apuN9tKCPat5dhGPvWCz6WIEeofC0s,41,[no],Are you the first person in your family to gra...,parent
9,1.0,scale,2017-01-15 01:07:52.71889 UTC,4536217521618944,Katie,[answer],wlbqIQAZcupMEcczJxbCGwX2QiNKJogQ,434,[yes],Does {kid_name} usually share with others?,child


In [117]:
df = df[df.columns[df.isnull().all()]]
df

""
0
1
2
3
4
5
6
7
8
9


In [14]:
df['learner'].drop_duplicates()

0        4535845151309824
3        4536217521618944
23       4536398325481472
25       4551846551093248
30       4554084669456384
42       4563324725362688
44       4570824568733696
48       4576288907984896
54       4594617454428160
60       4597232309370880
62       4611123475120128
65       4613797645910016
67       4619529606922240
70       4623601126866944
75       4628968502198272
77       4631671437524992
79       4631909271339008
81       4640195836116992
147      4643024344711168
163      4643216242507776
166      4646135276765184
168      4649037902905344
183      4649666666823680
219      4651483148582912
222      4654768731455488
224      4657658128760832
226      4657999083732992
229      4659795990675456
239      4664326778519552
241      4666370654470144
               ...       
32550    6577171304808448
32554    6577703167721472
32558    6579174856720384
32560    6580381545398272
32562    6582805790195712
32564    6583070106845184
32565    6592382661296128
32575    659

2734 in 'Learners' SQL

In [17]:
df['last_modified'] = pd.to_datetime(df['last_modified'])
least_recent_date = df['last_modified'].min()
recent_date = df['last_modified'].max()

In [18]:
recent_date

Timestamp('2018-11-28 03:55:30.195524+0000', tz='UTC')

In [21]:
latest = df.loc[df.groupby('name').last_modified.idxmax()]

In [23]:
latest.shape

(16120, 11)

In [28]:
lastest = df.sort_values('last_modified').groupby('name').tail(1)

In [26]:
lastest.shape

(16120, 11)